Concrete Compressive Strength Prediction using Neural Networks
- This Jupyter Notebook demonstrates the process of building and evaluating neural network models to predict concrete compressive strength using various techniques.

Data Preparation
- We start by importing necessary libraries and loading the concrete strength dataset. The dataset includes various features that affect concrete strength, such as cement content, water content, and age.

In [2]:
import pandas as pd
import numpy as np

import warnings
warnings.simplefilter('ignore', FutureWarning)

In [3]:
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [4]:
concrete_data.shape

(1030, 9)

In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [7]:
concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

In [8]:
predictors.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360


In [9]:
target.head()

0    79.99
1    61.89
2    40.27
3    41.05
4    44.30
Name: Strength, dtype: float64

In [10]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [11]:
n_cols = predictors_norm.shape[1] # number of predictors

In [14]:
import keras

In [15]:
from keras.models import Sequential
from keras.layers import Dense

In [16]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense

Part A: Baseline Model
- In this section, we build a baseline neural network model with the following specifications:

1. One hidden layer with 10 nodes and ReLU activation
2. Adam optimizer
3. Mean Squared Error (MSE) as the loss function

We perform the following steps:

1. Split the data into training (70%) and testing (30%) sets
2. Train the model for 50 epochs
3. Evaluate the model on the test set and compute MSE
4. Repeat steps 1-3 fifty times
5. Calculate the mean and standard deviation of the MSEs

In [17]:
def baseline_model():
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

In [18]:
def run_experiment():
    mse_list = []
    for _ in range(50):
        # Split the data
        X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.3, random_state=None)
        
        # Create and train the model
        model = baseline_model()
        model.fit(X_train, y_train, epochs=50, verbose=0)
        
        # Evaluate the model
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        mse_list.append(mse)
    
    return mse_list

In [19]:
mse_results = run_experiment()

C:\Users\scott\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


C:\Users\scott\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


C:\Users\scott\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


C:\Users\scott\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


C:\Users\scott\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


In [20]:
mean_mse = np.mean(mse_results)
std_mse = np.std(mse_results)

In [21]:
print(f"Mean MSE: {mean_mse:.4f}")
print(f"Standard Deviation of MSE: {std_mse:.4f}")

Mean MSE: 325.4349
Standard Deviation of MSE: 370.0503


Data Normalization
- We normalize the data by subtracting the mean and dividing by the standard deviation for each feature.

Experiment with Normalized Data
- We use the same model architecture and training process as in Part A, but with the normalized data:

In [22]:
def run_normalized_experiment():
    mse_list = []
    for _ in range(50):
        # Split the normalized data
        X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=None)
        
        # Create and train the model
        model = baseline_model()
        model.fit(X_train, y_train, epochs=50, verbose=0)
        
        # Evaluate the model
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        mse_list.append(mse)
    
    return mse_list

In [23]:
mse_results_normalized = run_normalized_experiment()

C:\Users\scott\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


C:\Users\scott\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


C:\Users\scott\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


C:\Users\scott\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


C:\Users\scott\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


In [24]:
mean_mse_normalized = np.mean(mse_results_normalized)
std_mse_normalized = np.std(mse_results_normalized)

In [25]:
print(f"Mean MSE (Normalized): {mean_mse_normalized:.4f}")
print(f"Standard Deviation of MSE (Normalized): {std_mse_normalized:.4f}")

Mean MSE (Normalized): 366.3711
Standard Deviation of MSE (Normalized): 99.7951


Comparison with Part A
- We compare the results of the normalized data model with the baseline model:

In [26]:
print(f"\nComparison:")
print(f"Part A Mean MSE: {mean_mse:.4f}")
print(f"Part B Mean MSE: {mean_mse_normalized:.4f}")


Comparison:
Part A Mean MSE: 325.4349
Part B Mean MSE: 366.3711


This comparison allows us to assess the impact of data normalization on the model's performance. Normalization can often lead to improved model convergence and potentially better results, especially for neural networks with multiple layers or when features have different scales.

Part C: Increased Number of Epochs
-In this section, we explore the impact of increasing the number of training epochs on our model's performance. We'll use the normalized data from Part B and increase the number of epochs from 50 to 100.

Experiment with Increased Epochs
- We modify our experiment function to train the model for 100 epochs instead of 50:

In [27]:
def run_increased_epochs_experiment():
    mse_list = []
    for _ in range(50):
        # Split the normalized data
        X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=None)
        
        # Create and train the model with 100 epochs
        model = baseline_model()
        model.fit(X_train, y_train, epochs=100, verbose=0)
        
        # Evaluate the model
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        mse_list.append(mse)
    
    return mse_list

In [28]:
mse_results_increased_epochs = run_increased_epochs_experiment()

C:\Users\scott\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


C:\Users\scott\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


C:\Users\scott\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


C:\Users\scott\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


C:\Users\scott\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


In [29]:
mean_mse_increased_epochs = np.mean(mse_results_increased_epochs)
std_mse_increased_epochs = np.std(mse_results_increased_epochs)

Comparison with Previous Results
- We compare the results of the increased epochs model with the previous models:

In [30]:
print(f"Mean MSE (Increased Epochs): {mean_mse_increased_epochs:.4f}")
print(f"Standard Deviation of MSE (Increased Epochs): {std_mse_increased_epochs:.4f}")

Mean MSE (Increased Epochs): 164.0823
Standard Deviation of MSE (Increased Epochs): 12.2189


In [31]:
print(f"\nComparison:")
print(f"Part A Mean MSE: {mean_mse:.4f}")
print(f"Part B Mean MSE: {mean_mse_normalized:.4f}")
print(f"Part C Mean MSE: {mean_mse_increased_epochs:.4f}")


Comparison:
Part A Mean MSE: 325.4349
Part B Mean MSE: 366.3711
Part C Mean MSE: 164.0823


Increasing the number of epochs allows the model to train for a longer period, potentially learning more complex patterns in the data. However, it's important to note that more epochs don't always lead to better performance. In some cases, it can lead to overfitting, where the model performs well on the training data but poorly on unseen data.

By comparing the results from Parts A, B, and C, we can observe how data normalization and increased training time affect the model's performance. This comparison helps us understand the trade-offs between these different approaches and guides us in selecting the most effective model for our concrete strength prediction task.

Part D: Increased Model Complexity
- In this final section, we explore the impact of increasing the model's complexity by adding more hidden layers. We'll use the normalized data and keep the number of epochs at 50, but increase the number of hidden layers from one to three.

Complex Model Architecture
- We define a new model with three hidden layers, each containing 10 nodes:

In [32]:
def complex_model():
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

Experiment with Complex Model
- We run the experiment using this new, more complex model:

In [33]:
def run_complex_model_experiment():
    mse_list = []
    for _ in range(50):
        # Split the normalized data
        X_train, X_test, y_train, y_test = train_test_split(predictors_norm, target, test_size=0.3, random_state=None)
        
        # Create and train the complex model
        model = complex_model()
        model.fit(X_train, y_train, epochs=50, verbose=0)
        
        # Evaluate the model
        y_pred = model.predict(X_test)
        mse = mean_squared_error(y_test, y_pred)
        mse_list.append(mse)
    
    return mse_list

In [34]:
mse_results_complex = run_complex_model_experiment()

C:\Users\scott\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


C:\Users\scott\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


C:\Users\scott\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


C:\Users\scott\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


C:\Users\scott\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


In [35]:
mean_mse_complex = np.mean(mse_results_complex)
std_mse_complex = np.std(mse_results_complex)

In [36]:
print(f"Mean MSE (Complex Model): {mean_mse_complex:.4f}")
print(f"Standard Deviation of MSE (Complex Model): {std_mse_complex:.4f}")

Mean MSE (Complex Model): 130.9415
Standard Deviation of MSE (Complex Model): 15.4480


Final Comparison
- We compare the results of all four experiments:

In [37]:
print(f"\nFinal Comparison:")
print(f"Part A Mean MSE: {mean_mse:.4f}")
print(f"Part B Mean MSE: {mean_mse_normalized:.4f}")
print(f"Part C Mean MSE: {mean_mse_increased_epochs:.4f}")
print(f"Part D Mean MSE: {mean_mse_complex:.4f}")


Final Comparison:
Part A Mean MSE: 325.4349
Part B Mean MSE: 366.3711
Part C Mean MSE: 164.0823
Part D Mean MSE: 130.9415


Throughout this notebook, we've explored various techniques to improve our neural network's performance in predicting concrete compressive strength:

- Part A: We established a baseline model with a single hidden layer.
- Part B: We normalized the input data, which can help with model convergence.
- Part C: We increased the number of training epochs from 50 to 100.
- Part D: We increased the model complexity by adding more hidden layers.

By comparing the Mean Squared Error (MSE) across these experiments, we can draw conclusions about the effectiveness of each approach:

- Data normalization (Part B) worsened the model's performance compared to the baseline.
- Increasing the number of epochs (Part C) improved the performance further.
- Adding more hidden layers (Part D) resulted in the best performance.

These experiments demonstrate the importance of iterative model development and the impact of various techniques on neural network performance. When working on similar regression problems, it's crucial to experiment with different architectures, hyperparameters, and preprocessing techniques to find the optimal model for your specific dataset and problem.